In [1]:
from sklearn.metrics import accuracy_score, confusion_matrix, roc_curve, auc
from sklearn.preprocessing import label_binarize
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier,BaggingClassifier,ExtraTreesClassifier,GradientBoostingClassifier
import numpy as np
from sklearn import svm
from sklearn.multiclass import OneVsRestClassifier

In [2]:
%matplotlib inline
import os
import operator
import numpy as np
import pandas as pd
from football_loader import metrics


def load_league_csv(league, start_year=2005):
    df = None
    files = os.listdir(f'fulldata/{league}')
    files.sort()
    for file in files:
        year = int(file.strip('.csv'))
        if year < start_year:
            continue
        df_year = pd.read_csv(f'fulldata/{league}/' + file,
                              engine='python',
                              # skiprows=1,
                              # index_col=None,
                              # names=['Div', 'Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTHG', 'HTAG', 'HTR', 'Referee', 'HS', 'AS', 'HST', 'AST', 'HF', 'AF', 'HC', 'AC', 'HY', 'AY', 'HR', 'AR', 'B365H', 'B365D', 'B365A']
                              )
        df_year.reset_index(drop=True, inplace=True)
        df_year['Year'] = year
        df_year['Match'] = df_year.index + 1

        if df is None:
            df = df_year
        else:
            df = df.append(df_year, ignore_index=True, sort=False)

    # print(len(df))
    # print(df.shape)

    # remove unused columns
    # df_league = None
    df.reset_index(inplace=True)
    # df = df[['Year', 'Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTHG', 'HTAG', 'HTR', 'HS', 'AS', 'HST', 'AST', 'HF', 'AF', 'HC', 'AC', 'HY', 'AY', 'HR', 'AR',
    #          "B365H", "B365D", "B365A"]]
    return df


def make_features(df, teams):
    # df_league = None
    ret = {}

    # if teams is None:
    #     teams = np.unique(df.loc[df['Year'] == predict_year, 'HomeTeam'].values)
    #     teams.sort()
    for team in teams:

        df_team = df[(df['HomeTeam'] == team) | (df['AwayTeam'] == team)]
        all = pd.DataFrame(
            data={
                'Year': df_team['Year'],
                'Date': df_team['Date'],
                'Team': team,
                'HomeMatch': df_team['HomeTeam'] == team
            }
        )
        all['Opponent'] = np.where(all['HomeMatch'], df_team['AwayTeam'], df_team['HomeTeam'])
        # X['HalfTimeGoals'] = np.where(X['HomeMatch'], df_team['HTHG'], df_team['HTAG'])
        # X['HalfTimeOpponentGoals'] = np.where(X['HomeMatch'], df_team['HTAG'], df_team['HTHG'])
        # X['HalfTimeLead'] = X['HalfTimeGoals'] > X['HalfTimeOpponentGoals']
        # X['HalfTimeLeadMoreThanTwo'] = (X['HalfTimeGoals'] - X['HalfTimeOpponentGoals']) > 2
        # X['FullTimeGoals'] = np.where(X['HomeMatch'], ath_madrid['FTHG'], ath_madrid['FTAG'])
        # X['FullTimeOpponentGoals'] = np.where(X['HomeMatch'], ath_madrid['FTAG'], ath_madrid['FTHG'])
        all['FTR'] = df_team['FTR']
        # all['Won'] = np.where(all['HomeMatch'], df_team['FTR'] == 'H', df_team['FTR'] == 'A')
        all['Won'] = np.where(df_team['FTR'] == '', False, np.where(all['HomeMatch'], df_team['FTR'] == 'H', df_team['FTR'] == 'A'))
        all['Draw'] = np.where(df_team['FTR'] == '', False, df_team['FTR'] == 'D')
        all['Lost'] = np.where(df_team['FTR'] == '', False, np.where(all['HomeMatch'], df_team['FTR'] == 'A', df_team['FTR'] == 'H'))
        all['Result'] = np.where(df_team['FTR'] == '', '', np.where(all['Won'], 'Win', (np.where(all['Lost'], 'Lose', 'Draw'))))
        # X['SumGoals'] = X.groupby('Opponent')['FullTimeGoals'].transform(sum)
        all['B365Max'] = np.maximum(np.maximum(df_team['B365H'], df_team['B365A']), df_team['B365D'])
        all['B365Min'] = np.minimum(np.minimum(df_team['B365H'], df_team['B365A']), df_team['B365D'])
        all['B365Say'] = np.where(all['HomeMatch'],
                                  # home match
                                  np.where(all['B365Max'] == df_team['B365H'], -1,
                                           np.where(all['B365Max'] == df_team['B365A'], 1,
                                                    0)),
                                  # away match
                                  np.where(all['B365Max'] == df_team['B365H'], 1,
                                           np.where(all['B365Max'] == df_team['B365A'], -1,
                                                    0))
                                  )
        # all['B365Diff'] = np.where(all['B365Say'] == 1, all['B365Max'] - all['B365Min'],
        #                            all['B365Min'] - all['B365Max'])
        all['B365Diff'] = np.where(all['B365Say'] == 1, all['B365Min'] - all['B365Max'],
                                   np.where(all['B365Say'] == -1, all['B365Max'] - all['B365Min'],
                                            # draw
                                            np.where(all['HomeMatch'],
                                            df_team['B365A'] - df_team['B365H'],
                                            df_team['B365H'] - df_team['B365A']
                                            ))) * -1
        all['Corners'] = np.where(all['HomeMatch'], df_team['HC'], df_team['AC'])
        all['Shots'] = np.where(all['HomeMatch'], df_team['HS'], df_team['AS'])
        all['ShotsOnTarget'] = np.where(all['HomeMatch'], df_team['HST'], df_team['AST'])
        all['Points'] = np.where(all['Won'], 3,
                                 np.where(all['Draw'], 1, 0)
                                  )
        all['AdjustedPoints'] = np.where(all['HomeMatch'],
                                  # home match
                                         np.where(all['Won'], 1,
                                                  np.where(all['Draw'], 0, -1)
                                                  )
                                         ,
                                  # away match
                                         np.where(all['Won'], 1.5,
                                                  np.where(all['Draw'], 0.5, 0)
                                                  )
                                  )
        all['Goals'] = np.where(all['HomeMatch'], df_team['FTHG'], df_team['FTAG'])
        all['Conceded'] = np.where(all['HomeMatch'], df_team['FTAG'], df_team['FTHG'])

        # find number of times won against this opponent in last 5 meetings
        for key, groupByOpponent in all.groupby('Opponent'):
            # keep index as new a column, will be restored and assigned back to X later
            idx = groupByOpponent.index

            # make match day an index because rolling need an index date
            xx = groupByOpponent.set_index('Date')
            xx['idx'] = idx
            # shift to exclude self
            xx['Last5AgainstThisOpponentWon'] = xx['Won'].rolling(6).apply(lambda x: np.nansum(x.shift()), raw=False)
            xx['Last5AgainstThisOpponentDraw'] = xx['Draw'].rolling(6).apply(lambda x: np.nansum(x.shift()), raw=False)
            # xx['Last5AgainstThisOpponentLost'] = xx['Lost'].rolling(6).apply(lambda x: np.nansum(x.shift()), raw=False)

            xx['Last3AgainstThisOpponentWon'] = xx['Won'].rolling(4).apply(lambda x: np.nansum(x.shift()), raw=False)
            xx['Last3AgainstThisOpponentDraw'] = xx['Draw'].rolling(4).apply(lambda x: np.nansum(x.shift()), raw=False)

            xx['LastAgainstThisOpponentWon'] = xx['Won'].rolling(2).apply(lambda x: np.nansum(x.shift()), raw=False)
            xx['LastAgainstThisOpponentDraw'] = xx['Draw'].rolling(2).apply(lambda x: np.nansum(x.shift()), raw=False)
            # xx['LastThisOpponentLost'] = xx['Lost'].rolling(2).apply(lambda x: np.nansum(x.shift()), raw=False)

            # restore index
            xx = xx.set_index('idx')

            # assign back to the big dataframe
            all.loc[xx.index, 'Last5AgainstThisOpponentWon'] = xx['Last5AgainstThisOpponentWon']
            all.loc[xx.index, 'Last5AgainstThisOpponentDraw'] = xx['Last5AgainstThisOpponentDraw']
            # X.loc[xx.index, 'Last5AgainstThisOpponentLost'] = xx['Last5AgainstThisOpponentLost']
            all.loc[xx.index, 'Last3AgainstThisOpponentWon'] = xx['Last3AgainstThisOpponentWon']
            all.loc[xx.index, 'Last3AgainstThisOpponentDraw'] = xx['Last3AgainstThisOpponentDraw']
            all.loc[xx.index, 'LastAgainstThisOpponentWon'] = xx['LastAgainstThisOpponentWon']
            all.loc[xx.index, 'LastAgainstThisOpponentDraw'] = xx['LastAgainstThisOpponentDraw']
            # X.loc[xx.index, 'LastThisOpponentLost'] = xx['LastThisOpponentLost']

        # stats by year/season
        for year, groupByYear in all.groupby('Year'):
            # print(year)
            # keep index as new a column, will be restored and assigned back to X later
            idx = groupByYear.index

            # make match day an index because rolling need an index date
            xx = groupByYear.set_index('Date')
            xx['idx'] = idx

            # shift to exclude self
            xx['CornersSoFar'] = np.nancumsum(xx['Corners'].shift())
            xx['ShotsSoFar'] = np.nancumsum(xx['Shots'].shift())
            xx['ShotsOnTargetSoFar'] = np.nancumsum(xx['ShotsOnTarget'].shift())
            xx['GoalsSoFar'] = np.nancumsum(xx['Goals'].shift())
            xx['ConcededSoFar'] = np.nancumsum(xx['Conceded'].shift())

            xx['HomeWonNum'] = np.where(xx['HomeMatch'] & xx['Won'], 1, 0)
            xx['HomeWonSoFar'] = np.nancumsum(xx['HomeWonNum'].shift())
            xx['AwayWonNum'] = np.where((xx['HomeMatch'] == False) & xx['Won'], 1, 0)
            xx['AwayWonSoFar'] = np.nancumsum(xx['AwayWonNum'].shift())

            xx['PointsSoFar'] = np.nancumsum(xx['Points'].shift())
            xx['AdjustedPointsSoFar'] = np.nancumsum(xx['AdjustedPoints'].shift())

            # restore index
            xx = xx.set_index('idx')

            # assign back to the big dataframe
            # all.loc[xx.index, 'CornersSoFar'] = xx['CornersSoFar']
            # all.loc[xx.index, 'ShotsSoFar'] = xx['ShotsSoFar']
            # all.loc[xx.index, 'ShotsOnTargetSoFar'] = xx['ShotsOnTargetSoFar']
            # all.loc[xx.index, 'GoalsSoFar'] = xx['GoalsSoFar']
            # all.loc[xx.index, 'ConcededSoSoFar'] = xx['ConcededSoSoFar']
#             all.loc[xx.index, 'HomeWonSoFar'] = xx['HomeWonSoFar']
#             all.loc[xx.index, 'AwayWonSoFar'] = xx['AwayWonSoFar']
            all.loc[xx.index, 'PointsSoFar'] = xx['PointsSoFar']
            all.loc[xx.index, 'AdjustedPointsSoFar'] = xx['AdjustedPointsSoFar']

        # find recent forms
        idx = all.index
        xx = all.set_index('Date')
        xx['idx'] = idx
        xx['Last5Won'] = xx['Won'].rolling(6).apply(lambda x: np.nansum(x.shift()), raw=False)
        xx['Last5Draw'] = xx['Draw'].rolling(6).apply(lambda x: np.nansum(x.shift()), raw=False)
        # xx['Last5Lost'] = xx['Lost'].rolling(6).apply(lambda x: np.nansum(x.shift()), raw=False)
        xx['Last3Won'] = xx['Won'].rolling(4).apply(lambda x: np.nansum(x.shift()), raw=False)
        xx['Last3Draw'] = xx['Draw'].rolling(4).apply(lambda x: np.nansum(x.shift()), raw=False)
        xx['LastWon'] = xx['Won'].rolling(2).apply(lambda x: np.nansum(x.shift()), raw=False)
        xx['LastDraw'] = xx['Draw'].rolling(2).apply(lambda x: np.nansum(x.shift()), raw=False)

        # restore index
        xx = xx.set_index('idx')
        # assign back to the big dataframe
        all.loc[xx.index, 'Last5Won'] = xx['Last5Won']
        all.loc[xx.index, 'Last5Draw'] = xx['Last5Draw']
        all.loc[xx.index, 'Last3Won'] = xx['Last3Won']
        all.loc[xx.index, 'Last3Draw'] = xx['Last3Draw']
        all.loc[xx.index, 'LastWon'] = xx['LastWon']
        all.loc[xx.index, 'LastDraw'] = xx['LastDraw']
        # X.loc[xx.index, 'Last5Lost'] = xx['Last5Lost']

        # replace nan with 0
        # TODO: better way to handle nan
        # all.loc[np.isnan(all['FTR']), 'FTR'] = ''
        all.loc[np.isnan(all['Last5AgainstThisOpponentWon']), 'Last5AgainstThisOpponentWon'] = 0
        all.loc[np.isnan(all['Last5AgainstThisOpponentDraw']), 'Last5AgainstThisOpponentDraw'] = 0
        # X.loc[np.isnan(X['Last5AgainstThisOpponentLost']), 'Last5AgainstThisOpponentLost'] = 0
        all.loc[np.isnan(all['Last3AgainstThisOpponentWon']), 'Last3AgainstThisOpponentWon'] = 0
        all.loc[np.isnan(all['Last3AgainstThisOpponentDraw']), 'Last3AgainstThisOpponentDraw'] = 0
        all.loc[np.isnan(all['LastAgainstThisOpponentWon']), 'LastAgainstThisOpponentWon'] = 0
        all.loc[np.isnan(all['LastAgainstThisOpponentDraw']), 'LastAgainstThisOpponentDraw'] = 0
        # X.loc[np.isnan(X['LastThisOpponentLost']), 'LastThisOpponentLost'] = 0
        all.loc[np.isnan(all['Last5Won']), 'Last5Won'] = 0
        all.loc[np.isnan(all['Last5Draw']), 'Last5Draw'] = 0
        # X.loc[np.isnan(X['Last5Lost']), 'Last5Lost'] = 0
        all.loc[np.isnan(all['Last3Won']), 'Last3Won'] = 0
        all.loc[np.isnan(all['Last3Draw']), 'Last3Draw'] = 0
        all.loc[np.isnan(all['LastWon']), 'LastWon'] = 0
        all.loc[np.isnan(all['LastDraw']), 'LastDraw'] = 0
        all.loc[np.isnan(all['B365Diff']), 'B365Diff'] = 0

        # restrict training data (too old data may not be irrelevance)
        X = all
        Y = X[['Result']]
        # del X['Result']
        # X = all.loc[(all['Year'] >= train_year) & (all['Year'] < predict_year)]
        # Y = all[['Result']]

        # split data into train - test sets
        # x_train = X[(X['Year'] < predict_year)]
        # y_train = Y[(X['Year'] < predict_year)]
        # x_test = X[(X['Year'] >= predict_year)]
        # y_test = Y[(X['Year'] >= predict_year)]
        # X['Predict'] = ''
        close_leaks(X)
        ret[team] = [X, Y]
    return ret


# call this after you've split data
def close_leaks(X):
    # remove duplicate features
#     del X['LastWon']
#     del X['LastDraw']

    # prevent future leaks
    # result = pd.DataFrame(X['Result'])
    del X['Result']
    del X['Lost']
    del X['Draw']
    del X['Won']
    del X['FTR']
    del X['Date']
    del X['Opponent']
    del X['Team']
    del X['B365Max']
    del X['B365Min']
    del X['Corners']
    del X['Shots']
    del X['ShotsOnTarget']
    del X['Points']
    del X['AdjustedPoints']
    del X['Goals']
    del X['Conceded']


In [3]:
league = 'english'
validate_year = 2016
test_year = 2017
train_year = 2005
df = load_league_csv(league)

In [4]:
teams = df.loc[(df['Year'] == validate_year) | (df['Year'] == test_year), 'HomeTeam']
teams = teams.unique()
teams.sort()
print(teams)
print(len(teams))

['Arsenal' 'Bournemouth' 'Brighton' 'Burnley' 'Chelsea' 'Crystal Palace'
 'Everton' 'Huddersfield' 'Hull' 'Leicester' 'Liverpool' 'Man City'
 'Man United' 'Middlesbrough' 'Newcastle' 'Southampton' 'Stoke'
 'Sunderland' 'Swansea' 'Tottenham' 'Watford' 'West Brom' 'West Ham']
23


In [5]:
teams = make_features(df, teams)
classes = ['Draw', 'Lose', 'Win']

In [6]:
print('X')
print(teams[next(iter(teams))][0].count()) # X
print('Y')
print(teams[next(iter(teams))][1].count()) # Y
print(len(teams), 'teams')

X
Year                            494
HomeMatch                       494
B365Say                         494
B365Diff                        494
Last5AgainstThisOpponentWon     494
Last5AgainstThisOpponentDraw    494
Last3AgainstThisOpponentWon     494
Last3AgainstThisOpponentDraw    494
LastAgainstThisOpponentWon      494
LastAgainstThisOpponentDraw     494
PointsSoFar                     494
AdjustedPointsSoFar             494
Last5Won                        494
Last5Draw                       494
Last3Won                        494
Last3Draw                       494
LastWon                         494
LastDraw                        494
dtype: int64
Y
Result    494
dtype: int64
23 teams


In [93]:
total = None
for team in teams:
# for team in ['Arsenal']:
    X = teams[team][0]
    Y = teams[team][1]
    # split data into train - validate - test sets
    x_train = X[(X['Year'] >= train_year) & (X['Year'] < validate_year)]
    y_train = Y[(X['Year'] >= train_year) & (X['Year'] < validate_year)]
    x_validate = X[(X['Year'] >= validate_year) & (X['Year'] < test_year)]
    y_validate = Y[(X['Year'] >= validate_year) & (X['Year'] < test_year)]
    x_test = X[(X['Year'] >= test_year)]
    y_test = Y[(X['Year'] >= test_year)]
    if len(x_train) <= 0 or len(x_test) <= 0 or len(x_validate) <= 0:
        print(f'skip {team}')
        continue
    
    validate_accuracies = {}
    test_accuracies = {}
    
    lr = LogisticRegression()
    lr.fit(x_train, y_train['Result'])
    y_validate_pred = lr.predict(x_validate)
    validate_accuracies['LogisticRegression'] = accuracy_score(y_validate, y_validate_pred) * 100
    y_test_pred = lr.predict(x_test)
    test_accuracies['LogisticRegression'] = accuracy_score(y_test, y_test_pred) * 100
    y_validate_prob = lr.predict_proba(x_validate)
    y_test_prob = lr.predict_proba(x_test)
#     print(y_validate_prob)
#     print(y_test_prob)
    prob = np.vstack((y_validate_prob, y_test_prob))
#     print(prob)
#     print(prob[:,0])
#     print(np.max(prob, axis=1))
#     print(np.min(prob, axis=1))
    d = pd.DataFrame(
        data={
            'Year': x_validate['Year'].append(x_test['Year']),
            'Team': team,
            'Predict': np.append(y_validate_pred, y_test_pred),
            'Actual': y_validate['Result'].append(y_test['Result']),
            'DrawProb': prob[:,0],
            'LoseProb': prob[:,1],
            'WinProb': prob[:,2],
            'MaxProb': np.max(prob, axis=1),
            'MinProb': np.min(prob, axis=1)
        },
        index=np.append(y_validate.index, y_test.index)
#         index=pd.MultiIndex.from_tuples(
#             list(zip(np.repeat('LogisticRegression', len(y_test.index)), y_test.index.values)),
#             names=['predictor', 'match_id'])
    )
    if total is None:
        total = d
    else:
        total = total.append(d)
    for (k, v) in validate_accuracies.items():
        print(f"{k}: {team} validation accuracy are: {v}")
#         cm = confusion_matrix(y_validate, y_validate_pred, labels=classes)
#         metrics.plot_confusion_matrix(cm, classes, title=team)
        print(f"{k}: {team} test accuracy are: {test_accuracies[k]}")
#         cm = confusion_matrix(y_test, y_test_pred, labels=classes)
#         metrics.plot_confusion_matrix(cm, classes, title=team)
#     print(lr.coef_)
    



LogisticRegression: Arsenal validation accuracy are: 63.1578947368421
LogisticRegression: Arsenal test accuracy are: 63.1578947368421
LogisticRegression: Bournemouth validation accuracy are: 55.26315789473685
LogisticRegression: Bournemouth test accuracy are: 50.0
skip Brighton
LogisticRegression: Burnley validation accuracy are: 52.63157894736842
LogisticRegression: Burnley test accuracy are: 34.21052631578947
LogisticRegression: Chelsea validation accuracy are: 78.94736842105263
LogisticRegression: Chelsea test accuracy are: 52.63157894736842
LogisticRegression: Crystal Palace validation accuracy are: 44.73684210526316
LogisticRegression: Crystal Palace test accuracy are: 55.26315789473685
LogisticRegression: Everton validation accuracy are: 42.10526315789473
LogisticRegression: Everton test accuracy are: 68.42105263157895
skip Huddersfield
skip Hull
LogisticRegression: Leicester validation accuracy are: 44.73684210526316
LogisticRegression: Leicester test accuracy are: 42.1052631578

In [94]:
# for key, groupByPredictor in total.groupby('predictor'):
#     print(f"{key} overall accuracy is ", accuracy_score(groupByPredictor['actual'], groupByPredictor['pred'])*100)
print(f"Overall accuracy is ", accuracy_score(total['Actual'], total['Predict'])*100)

Overall accuracy is  53.40557275541795


In [95]:
total

,Year,Team,Predict,Actual,DrawProb,LoseProb,WinProb,MaxProb,MinProb
4188,2016,Arsenal,Win,Lose,0.316564,0.328489,0.354947,0.354947,0.316564
4193,2016,Arsenal,Win,Draw,0.110749,0.275769,0.613481,0.613481,0.110749
4208,2016,Arsenal,Win,Win,0.113264,0.112297,0.774439,0.774439,0.112297
4211,2016,Arsenal,Win,Win,0.210044,0.168096,0.621860,0.621860,0.168096
4223,2016,Arsenal,Win,Win,0.243385,0.129176,0.627439,0.627439,0.129176
4231,2016,Arsenal,Draw,Win,0.370888,0.310070,0.319042,0.370888,0.310070
4247,2016,Arsenal,Win,Win,0.071672,0.108562,0.819767,0.819767,0.071672
4251,2016,Arsenal,Win,Win,0.177462,0.110696,0.711842,0.711842,0.110696
4261,2016,Arsenal,Win,Draw,0.055047,0.027553,0.917400,0.917400,0.027553
4274,2016,Arsenal,Win,Win,0.185685,0.178486,0.635829,0.635829,0.178486


In [96]:
chart_data = {}
chart_data_all = None
for team, dataset in teams.items():
    chart_data_team = pd.DataFrame(dataset[0])
    chart_data_team = chart_data_team[(chart_data_team['Year'] >= validate_year) | (chart_data_team['Year'] >= test_year)]
    chart_data_team['Actual'] = dataset[1]['Result']
    i = total[total.index.isin(chart_data_team.index)]
    chart_data_team.loc[i.index, 'Predict'] = i['Predict']
    chart_data_team['Point'] = np.where(chart_data_team['Actual'] == 'Win', 3,
                                 np.where(chart_data_team['Actual'] == 'Draw', 1, 0)
                                  )
    chart_data_team['PointPredict'] = np.where(chart_data_team['Predict'] == 'Win', 3,
                                 np.where(chart_data_team['Predict'] == 'Draw', 1, 0)
                                  )
    for key, groupByYear in chart_data_team.groupby('Year'):
        chart_data_team.loc[groupByYear.index, 'MatchNo'] = range(1,len(groupByYear['Actual'])+1)
        chart_data_team.loc[groupByYear.index, 'TotalPoints'] = np.nancumsum(groupByYear['Point'])
        chart_data_team.loc[groupByYear.index, 'TotalPointsPredict'] = np.nancumsum(groupByYear['PointPredict'])
        
    chart_data_team['Team'] = team
    chart_data[team] = chart_data_team
    if chart_data_all is None:
        chart_data_all = chart_data_team
    else:
        chart_data_all = chart_data_all.append(chart_data_team, ignore_index=True, sort=False)
chart_data_all

,Year,HomeMatch,B365Say,B365Diff,Last5AgainstThisOpponentWon,Last5AgainstThisOpponentDraw,Last3AgainstThisOpponentWon,Last3AgainstThisOpponentDraw,LastAgainstThisOpponentWon,LastAgainstThisOpponentDraw,...,LastWon,LastDraw,Actual,Predict,Point,PointPredict,MatchNo,TotalPoints,TotalPointsPredict,Team
0,2016,True,0,-0.70,1.0,3.0,1.0,2.0,0.0,1.0,...,1.0,0.0,Lose,Draw,0,1,1.0,0.0,1.0,Arsenal
1,2016,False,0,-0.38,0.0,0.0,3.0,0.0,1.0,0.0,...,0.0,0.0,Draw,Lose,1,0,2.0,1.0,1.0,Arsenal
2,2016,False,1,3.27,0.0,0.0,3.0,0.0,1.0,0.0,...,0.0,1.0,Win,Lose,3,0,3.0,4.0,1.0,Arsenal
3,2016,True,1,4.93,1.0,2.0,0.0,1.0,0.0,1.0,...,1.0,0.0,Win,Lose,3,0,4.0,7.0,1.0,Arsenal
4,2016,False,1,4.40,4.0,1.0,2.0,1.0,1.0,0.0,...,1.0,0.0,Win,Win,3,3,5.0,10.0,4.0,Arsenal
5,2016,True,0,-0.70,0.0,1.0,0.0,1.0,0.0,0.0,...,1.0,0.0,Win,Win,3,3,6.0,13.0,7.0,Arsenal
6,2016,False,1,7.60,0.0,0.0,3.0,0.0,1.0,0.0,...,1.0,0.0,Win,Draw,3,1,7.0,16.0,8.0,Arsenal
7,2016,True,1,9.71,1.0,1.0,1.0,0.0,0.0,0.0,...,1.0,0.0,Win,Win,3,3,8.0,19.0,11.0,Arsenal
8,2016,True,1,13.75,1.0,3.0,1.0,2.0,1.0,0.0,...,1.0,0.0,Draw,Win,1,3,9.0,20.0,14.0,Arsenal
9,2016,False,1,7.60,3.0,2.0,1.0,2.0,0.0,1.0,...,0.0,1.0,Win,Win,3,3,10.0,23.0,17.0,Arsenal


In [97]:
fixtures = pd.DataFrame(df[df['Year'] == test_year])
# for team, dataset in teams.items():
#     df_team = pd.DataFrame(dataset[0])
#     df_team = df_team[df_team['Year'] == test_year]
#     df_team['Actual'] = dataset[1]['Result']
#     i = total[total.index.isin(df_team.index)]
#     df_team.loc[i.index, 'Predict'] = i['Predict']
#     df_team['MatchNo'] = range(1,len(df_team['Actual'])+1)
#     df_team['Team'] = team
#     df_team['Date'] = df.loc[df_team.index, 'Date']
#     df_team['Date'] = df.loc[df_team.index, 'Date']
#     df_team['Date'] = df.loc[df_team.index, 'Date']
                       
#     if fixtures is None:
#         fixtures = df_team
#     else:
#         fixtures = fixtures.append(df_team, ignore_index=True, sort=False)
fixtures.sort_values(['Date'], inplace=True)
# fixtures.dropna(subset=['Year'], inplace=True)
fixtures

,index,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,...,Match,BSH,BSD,BSA,PSH,PSD,PSA,PSCH,PSCD,PSCA
4773,4773,E0,01/01/18,Leicester,Huddersfield,3.0,0.0,H,0.0,0.0,...,213,NaN,NaN,NaN,1.65,4.08,6.01,1.83,3.52,5.23
4774,4774,E0,01/01/18,Stoke,Newcastle,0.0,1.0,A,0.0,0.0,...,214,NaN,NaN,NaN,2.36,3.26,3.44,2.31,3.28,3.54
4772,4772,E0,01/01/18,Everton,Man United,0.0,2.0,A,0.0,0.0,...,212,NaN,NaN,NaN,5.49,3.59,1.79,5.60,3.89,1.71
4771,4771,E0,01/01/18,Burnley,Liverpool,1.0,2.0,A,0.0,0.0,...,211,NaN,NaN,NaN,7.31,4.45,1.52,6.60,3.80,1.65
4770,4770,E0,01/01/18,Brighton,Bournemouth,2.0,2.0,D,1.0,1.0,...,210,NaN,NaN,NaN,2.46,3.11,3.44,2.47,3.05,3.48
4840,4840,E0,01/03/18,Arsenal,Man City,0.0,3.0,A,0.0,3.0,...,280,NaN,NaN,NaN,4.58,4.06,1.80,4.75,4.33,1.72
4873,4873,E0,01/04/18,Arsenal,Stoke,3.0,0.0,H,0.0,0.0,...,313,NaN,NaN,NaN,1.31,6.10,10.47,1.31,6.05,11.00
4874,4874,E0,01/04/18,Chelsea,Tottenham,1.0,3.0,A,1.0,1.0,...,314,NaN,NaN,NaN,2.43,3.40,3.18,2.30,3.43,3.40
4630,4630,E0,01/10/17,Newcastle,Liverpool,1.0,1.0,D,1.0,1.0,...,70,NaN,NaN,NaN,5.09,4.17,1.71,5.02,4.14,1.72
4629,4629,E0,01/10/17,Everton,Burnley,0.0,1.0,A,0.0,1.0,...,69,NaN,NaN,NaN,1.68,3.81,6.20,1.71,3.77,5.75


In [139]:
starting_balance = 1000
balance = starting_balance
print("Starting balance:", balance)
for date, groupByDate in fixtures.groupby('Date'):
    for _, match in groupByDate.iterrows():
#         print(match['index'])
        predictions = total[total.index == match['index']]
        if (len(predictions) == 1):
#             prediction = predictions[:,0]
#             print(predictions)
            team = predictions['Team'].values[0]
            pred = predictions['Predict'].values[0]
            act = predictions['Actual'].values[0]
            print(team, "Pred: ", pred, "Actual:", act)
            pay = -50
            print(date, "Pay:", pay)
            balance += pay
            odd = 0
            if pred == act:
#                 print(match)
                if pred == act == 'Draw':
                    odd = match['B365D']
                elif match['HomeTeam'] == team:
                    odd = match['B365H']
                else:
                    odd = match['B365A']
                receive = pay * odd * -1
                balance += receive
                print(date, "Receive:", receive, "Balance:", balance)
        else:
            print(predictions)

print("Ending balance:", balance)

Starting balance: 1000
Leicester Pred:  Win Actual: Win
01/01/18 Pay: -50
01/01/18 Receive: 83.0 Balance: 1033.0
Stoke Pred:  Win Actual: Lose
01/01/18 Pay: -50
      Year        Team Predict Actual  DrawProb  LoseProb   WinProb   MaxProb  \
4772  2017     Everton    Lose   Lose  0.143409  0.594829  0.261762  0.594829   
4772  2017  Man United     Win    Win  0.068517  0.446475  0.485008  0.485008   

       MinProb  
4772  0.143409  
4772  0.068517  
      Year       Team Predict Actual  DrawProb  LoseProb   WinProb   MaxProb  \
4771  2017    Burnley    Lose   Lose  0.228034  0.764275  0.007691  0.764275   
4771  2017  Liverpool     Win    Win  0.238874  0.137704  0.623422  0.623422   

       MinProb  
4771  0.007691  
4771  0.137704  
Bournemouth Pred:  Draw Actual: Draw
01/01/18 Pay: -50
01/01/18 Receive: 160.0 Balance: 1093.0
      Year      Team Predict Actual  DrawProb  LoseProb   WinProb   MaxProb  \
4840  2017   Arsenal    Lose   Lose  0.340123  0.403069  0.256807  0.403069   

      Year       Team Predict Actual  DrawProb  LoseProb   WinProb   MaxProb  \
4918  2017  Leicester    Draw   Lose  0.499104  0.129922  0.370974  0.499104   
4918  2017   West Ham    Lose    Win  0.125247  0.447191  0.427562  0.447191   

       MinProb  
4918  0.129922  
4918  0.125247  
      Year         Team Predict Actual  DrawProb  LoseProb   WinProb  \
4917  2017      Everton     Win   Draw  0.300928  0.231678  0.467395   
4917  2017  Southampton     Win   Draw  0.402549  0.062486  0.534964   

       MaxProb   MinProb  
4917  0.467395  0.231678  
4917  0.534964  0.062486  
      Year         Team Predict Actual  DrawProb  LoseProb   WinProb  \
4916  2017  Bournemouth     Win    Win  0.300816  0.048166  0.651018   
4916  2017      Swansea     Win   Lose  0.239371  0.267956  0.492674   

       MaxProb   MinProb  
4916  0.651018  0.048166  
4916  0.492674  0.239371  
      Year        Team Predict Actual  DrawProb  LoseProb   WinProb   MaxProb  \
4667  2017     Chelsea     Win 

4721  0.181858  
West Brom Pred:  Win Actual: Win
13/01/18 Pay: -50
13/01/18 Receive: 107.0 Balance: 799.5
      Year       Team Predict Actual  DrawProb  LoseProb   WinProb   MaxProb  \
4781  2017    Chelsea     Win   Draw  0.245032   0.03628  0.718688  0.718688   
4781  2017  Leicester    Lose   Draw  0.152518   0.83134  0.016142  0.831340   

       MinProb  
4781  0.036280  
4781  0.016142  
      Year            Team Predict Actual  DrawProb  LoseProb   WinProb  \
4782  2017         Burnley    Lose   Lose  0.115066  0.880498  0.004435   
4782  2017  Crystal Palace    Draw    Win  0.449898  0.332668  0.217434   

       MaxProb   MinProb  
4782  0.880498  0.004435  
4782  0.449898  0.217434  
West Ham Pred:  Lose Actual: Win
13/01/18 Pay: -50
Swansea Pred:  Win Actual: Draw
13/01/18 Pay: -50
      Year       Team Predict Actual  DrawProb  LoseProb   WinProb   MaxProb  \
4785  2017    Everton    Lose   Lose  0.415908  0.546997  0.037095  0.546997   
4785  2017  Tottenham     Win    

Arsenal Pred:  Win Actual: Win
16/12/17 Pay: -50
16/12/17 Receive: 62.5 Balance: 956.5
Burnley Pred:  Lose Actual: Draw
16/12/17 Pay: -50
      Year         Team Predict Actual  DrawProb  LoseProb   WinProb  \
4733  2017      Chelsea     Win    Win  0.304814  0.081784  0.613402   
4733  2017  Southampton    Lose   Lose  0.214174  0.667808  0.118018   

       MaxProb   MinProb  
4733  0.613402  0.081784  
4733  0.667808  0.118018  
      Year            Team Predict Actual  DrawProb  LoseProb   WinProb  \
4734  2017  Crystal Palace     Win    Win  0.364441  0.151741  0.483818   
4734  2017       Leicester    Draw   Lose  0.552213  0.025120  0.422667   

       MaxProb   MinProb  
4734  0.483818  0.151741  
4734  0.552213  0.025120  
      Year       Team Predict Actual  DrawProb  LoseProb   WinProb   MaxProb  \
4735  2017   Man City     Win    Win  0.146777  0.191907  0.661316  0.661316   
4735  2017  Tottenham    Lose   Lose  0.338132  0.463757  0.198111  0.463757   

       MinProb  

4650  0.216423  
      Year     Team Predict Actual  DrawProb  LoseProb   WinProb   MaxProb  \
4649  2017  Arsenal     Win    Win  0.294174  0.321346  0.384480  0.384480   
4649  2017  Everton    Lose   Lose  0.169463  0.416243  0.414294  0.416243   

       MinProb  
4649  0.294174  
4649  0.169463  
      Year       Team Predict Actual  DrawProb  LoseProb   WinProb   MaxProb  \
4741  2017    Arsenal    Draw   Draw  0.400996  0.341358  0.257645  0.400996   
4741  2017  Liverpool     Win   Draw  0.251797  0.182156  0.566047  0.566047   

       MinProb  
4741  0.257645  
4741  0.182156  
Everton Pred:  Win Actual: Win
23/04/18 Pay: -50
23/04/18 Receive: 114.5 Balance: 623.5
      Year         Team Predict Actual  DrawProb  LoseProb   WinProb  \
4615  2017   Man United     Win    Win  0.182505  0.224186  0.593309   
4615  2017  Southampton    Lose   Lose  0.381435  0.457843  0.160722   

       MaxProb   MinProb  
4615  0.593309  0.182505  
4615  0.457843  0.160722  
      Year         

      Year       Team Predict Actual  DrawProb  LoseProb   WinProb   MaxProb  \
4588  2017    Arsenal    Lose   Lose  0.183700  0.441003  0.375297  0.441003   
4588  2017  Liverpool    Draw    Win  0.407325  0.222200  0.370475  0.407325   

      MinProb  
4588   0.1837  
4588   0.2222  
      Year       Team Predict Actual  DrawProb  LoseProb   WinProb   MaxProb  \
4589  2017    Burnley    Lose   Draw  0.309739  0.559666  0.130595  0.559666   
4589  2017  Tottenham     Win   Draw  0.154413  0.068186  0.777401  0.777401   

       MinProb  
4589  0.130595  
4589  0.068186  
      Year       Team Predict Actual  DrawProb  LoseProb   WinProb   MaxProb  \
4590  2017      Stoke    Draw   Draw  0.501752  0.310094  0.188153  0.501752   
4590  2017  West Brom     Win   Draw  0.314348  0.271371  0.414281  0.414281   

       MinProb  
4590  0.188153  
4590  0.271371  
      Year     Team Predict Actual  DrawProb  LoseProb   WinProb   MaxProb  \
4587  2017  Chelsea     Win    Win  0.332509  0.1

4769  0.253060  
      Year            Team Predict Actual  DrawProb  LoseProb   WinProb  \
4768  2017  Crystal Palace    Lose   Draw  0.083617  0.783081  0.133303   
4768  2017        Man City     Win   Draw  0.173650  0.080172  0.746178   

       MaxProb   MinProb  
4768  0.783081  0.083617  
4768  0.746178  0.080172  
Ending balance: 777.0
